In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from skimage.feature import hog
from skimage.io import imread_collection

In [2]:
folders = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,24,
               33,34,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,
               71,72,73,74,75,76,77,78,79,80,81]

pixels_per_cell = 16
hog_features = []
label = []

for fldr in folders:
    col_dir = f'cw_faces/{fldr}/*'
    col = imread_collection(col_dir)
    for image in col:
        fd,hog_image = hog(image, orientations=8, pixels_per_cell=(pixels_per_cell, pixels_per_cell),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
        hog_features.append(fd)
        label.append(fldr)
        

In [3]:
#convert data to correct type
hog_features = np.float32(hog_features)
label = np.array(label)

# Shuffle Samples
shuffle = np.random.permutation(len(hog_features))
hog_features = hog_features[shuffle]
label = label[shuffle]    

#split data into train and test sets
X_train = hog_features[:750]
X_test = hog_features[750:]
y_train = label[:750]
y_test = label[750:]

In [4]:
#Build and train SVM model
svm_hog = svm.LinearSVC()
svm_hog.fit(X_train, y_train)

/Users/eijaz/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [5]:
#get predictions and get accuracy of SVM model on the test set
y_pred_svm = svm_hog.predict(X_test)

print("Accuracy: "+str(accuracy_score(y_test, y_pred_svm)))

Accuracy: 0.8722222222222222


In [7]:
#train final SVM model on all data
svm_hog.fit(hog_features, label)

/Users/eijaz/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [8]:
#save final SVM model
joblib.dump(svm_hog, 'svm_hog.pkl')

['svm_hog.pkl']

In [9]:
#one-hot encode target column
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [13]:
#create model
mlp_hog = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=2)

#add model layers
mlp_hog.add(Dense(256, activation='relu', input_shape=(n_cols,)))
mlp_hog.add(Dense(256, activation='relu'))
mlp_hog.add(Dense(256, activation='relu'))
mlp_hog.add(Dense(82, activation='softmax'))

In [14]:
#compile model
mlp_hog.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
#train model
mlp_hog.fit(X_train, y_train_cat, validation_data=(X_test,y_test_cat), epochs=30, callbacks=[early_stopping_monitor])

Train on 750 samples, validate on 180 samples
Epoch 1/30
750/750 [==============================] - 3s 4ms/step - loss: 3.4652 - acc: 0.3067 - val_loss: 1.8261 - val_acc: 0.6556
Epoch 2/30
750/750 [==============================] - 2s 3ms/step - loss: 1.1392 - acc: 0.7573 - val_loss: 1.0743 - val_acc: 0.7778
Epoch 3/30
750/750 [==============================] - 2s 3ms/step - loss: 0.5174 - acc: 0.8787 - val_loss: 1.0137 - val_acc: 0.7944
Epoch 4/30
750/750 [==============================] - 2s 3ms/step - loss: 0.2316 - acc: 0.9467 - val_loss: 0.9256 - val_acc: 0.8000
Epoch 5/30
750/750 [==============================] - 2s 3ms/step - loss: 0.1107 - acc: 0.9853 - val_loss: 0.8613 - val_acc: 0.8167
Epoch 6/30
750/750 [==============================] - 2s 3ms/step - loss: 0.0382 - acc: 0.9987 - val_loss: 0.8209 - val_acc: 0.8222
Epoch 7/30
750/750 [==============================] - 2s 3ms/step - loss: 0.0097 - acc: 1.0000 - val_loss: 0.7639 - val_acc: 0.8389
Epoch 8/30
750/750 [==========

In [19]:
#save MLP model
mlp_hog.save('mlp_hog.h5')